# COVID-19 Classification using 4-layer Convolutional Neural Networks.

In [1]:
! pip install opencv-python

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import numpy as np
import shutil
from sklearn import preprocessing
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import f1_score
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from glob import glob

In [3]:
base_dir = './data/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

In [4]:
import tensorflow as tf
 
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(3, activation='softmax')  
])


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 64)       0

In [6]:
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'categorical',
                                                         target_size = (150, 150),
                                                         shuffle=False)

Found 72 images belonging to 3 classes.
Found 36 images belonging to 3 classes.


In [11]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=1,
                              epochs=20,
                              validation_steps = 3,
                              verbose=2)

Epoch 1/20
1/1 - 1s - loss: 0.9513 - accuracy: 0.4000 - val_loss: 1.0171 - val_accuracy: 0.3889 - 929ms/epoch - 929ms/step
Epoch 2/20
1/1 - 0s - loss: 0.9645 - accuracy: 0.5000 - 491ms/epoch - 491ms/step
Epoch 3/20
1/1 - 0s - loss: 0.8929 - accuracy: 0.8000 - 471ms/epoch - 471ms/step
Epoch 4/20
1/1 - 1s - loss: 0.9341 - accuracy: 0.5500 - 744ms/epoch - 744ms/step
Epoch 5/20
1/1 - 0s - loss: 0.8009 - accuracy: 0.8000 - 499ms/epoch - 499ms/step
Epoch 6/20
1/1 - 0s - loss: 0.7248 - accuracy: 0.7000 - 463ms/epoch - 463ms/step
Epoch 7/20
1/1 - 1s - loss: 0.7217 - accuracy: 0.7000 - 511ms/epoch - 511ms/step
Epoch 8/20
1/1 - 0s - loss: 0.6585 - accuracy: 0.8333 - 433ms/epoch - 433ms/step
Epoch 9/20
1/1 - 0s - loss: 0.6317 - accuracy: 0.8500 - 471ms/epoch - 471ms/step
Epoch 10/20
1/1 - 0s - loss: 0.5411 - accuracy: 0.8500 - 422ms/epoch - 422ms/step
Epoch 11/20
1/1 - 1s - loss: 0.5404 - accuracy: 0.8500 - 523ms/epoch - 523ms/step
Epoch 12/20
1/1 - 0s - loss: 0.3787 - accuracy: 0.9167 - 255ms/ep

In [12]:
from sklearn.metrics import classification_report
from sklearn import preprocessing

y_pred = model.predict(validation_generator, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(validation_generator.classes, y_pred_bool))

2/2 [==============================] - 0s 86ms/step
              precision    recall  f1-score   support

           0       1.00      0.83      0.91        12
           1       0.67      1.00      0.80        12
           2       0.75      0.50      0.60        12

    accuracy                           0.78        36
   macro avg       0.81      0.78      0.77        36
weighted avg       0.81      0.78      0.77        36

